In [1]:
import pandas as pd
import os 
from ingredients import *
from parse_funcs import *
from mfyfunctions import *
from analysis_functions import *

In [2]:
pd.set_option('display.float_format', '{:.1f}'.format)
pd.set_option('display.max_colwidth', None)
# noinspection PyTypeChecker
threshold_date = pd.Timestamp('2022-09-30')

In [3]:
datasets_path = r'011/datasets/'
datasets = os.listdir(datasets_path) 

In [4]:
dataframes = {}

In [5]:
for dataset in datasets:
    if dataset.endswith('.csv'):
        dataset_path = os.path.join(datasets_path, dataset)
        df = pd.read_csv(dataset_path)
        dataset_name = datasets_names[dataset]
        globals()[dataset_name] = df
        dataframes[dataset_name] = df

In [6]:
dataframes.keys()

dict_keys(['scr', 'int', 'gc', 'ic', 'fu', 'pei', 'trw', 'td', 'cws', 'aws', 'psfs', 'ptnt', 'ptre', 'ptgc', 'ptic', 'ptfu'])

In [9]:
scr_date_columns = parse_date_columns(scr)
scr = parse_date_columns(scr, convert_dates=True)
scr23 = scr[(scr[scr_date_columns] > threshold_date).any(axis=1)]

In [11]:
scr23

,scsp,scspi,rid,sex,age,scloc,sc1,sc2,scre,srs,srsorg,mhpss,trw,outref,tmh,pei,sert
22,YQ,23,R0183,2.0,48.0,NaN,2021-12-17,NaT,2023-06-21,Outreach_Session,NaN,NaN,NaN,NaN,NaN,1.0,1.0
27,YQ,28,R0120,2.0,44.0,NaN,2021-11-29,NaT,2023-06-21,Outreach_Session,NaN,NaN,NaN,NaN,NaN,1.0,1.0
79,YQ,80,R0290,1.0,34.0,NaN,2022-04-07,NaT,2023-06-16,ADMSP_Beneficiary,NA_,NaN,NaN,NaN,NaN,1.0,1.0
89,YQ,90,R0298,1.0,36.0,NaN,2022-04-13,NaT,2023-06-20,Outreach_Session,NaN,NaN,NaN,NaN,NaN,1.0,1.0
97,YQ,98,R0337,2.0,53.0,NaN,2022-04-25,NaT,2023-06-19,ADMSP_Beneficiary,NA_,NaN,NaN,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,MT,42,R0934,2.0,33.0,SF,2023-09-25,NaT,NaT,IRM,NA_,0.0,0.0,0.0,1.0,1.0,2.0
484,AY,17,R0483,1.0,48.0,GZT,2022-11-09,NaT,NaT,Family_or_Friend,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,AY,23,R0593,1.0,36.0,GZT,2022-12-21,NaT,NaT,Family_or_Friend,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,AY,25,R0512,1.0,23.0,GZT,2023-01-05,NaT,NaT,Family_or_Friend,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
int[int['hrd'] == 1]

,Unnamed: 0,ntsp,ntspi,rid,fcid,nt,nt1,nt2,nt3,ntre,...,sts,vsva,vsvp,hrd,jrn,wov,stgbv,lgbti,other,vict
91,91,YQ,92,R0415,8110020102,2,2022-06-09,2022-07-07,2022-07-21,NaN,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,7.0
227,227,SA,102,R0592,8110050096,1,2023-01-02,2023-01-16,NaN,NaN,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,6.0
228,228,SA,103,R0725,8110050097,2,2023-07-14,2023-07-18,2023-07-25,NaN,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,5.0
244,244,IJ,14,R0733,8110080014,2,2023-05-31,2023-06-05,2023-06-21,NaN,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,7.0
248,248,LA,3,R0708,8110100003,2,2023-05-24,2023-06-01,2023-07-28,NaN,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,8.0


In [9]:
# Creating empty dataframe
visits_columns = ['rid', 'sex', 'age', 'visit','date']
visits = pd.DataFrame(columns=visits_columns)

In [10]:
for key, value in dataframes.items():
    date_columns = parse_date_columns(value)
    df = melt_and_categorize_dates(
        value,
        date_columns,
        sort_by_date=True,
        var_name='visit',
        value_name='date'
    )
    if key == 'scr':
        visits = visits.assign(
            rid=df['rid'].values,
            sex=df['sex'].values,
            age=df['age'].values,
            visit=df['visit'].values,
            date=df['date'].values
        )
    else:
        visits = pd.concat([visits, df[['rid', 'visit', 'date']]], ignore_index=True)